# ETL YELP

# 1. Importar librerias

In [1]:
import pandas as pd
import numpy as np
import re
import requests
import warnings
warnings.filterwarnings("ignore", message="Downcasting.*", category=FutureWarning)


# 2. Carga de Datos

In [2]:
dfBusinessYelp = pd.read_csv('DataSets/businessYelp.csv')

C:\Users\U ANGEL\AppData\Local\Temp\ipykernel_15140\874172018.py:1: DtypeWarning: Columns (14,15,16,17,18,19,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  dfBusinessYelp = pd.read_csv('DataSets/businessYelp.csv')


In [3]:
dfReviewYelp = pd.read_csv('DataSets/ReviewsYelp.csv')

# 3. Exploracion y Limpieza de Datos

## 3.1 Dataset Business Yelp

### 3.1.1. Exploración inicial de los datos

In [4]:
#Vizualizo las primeras filas del DataFrame
dfBusinessYelp.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state.1,postal_code.1,latitude.1,longitude.1,stars.1,review_count.1,is_open.1,attributes.1,categories.1,hours.1
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Obtener información sobre las columnas y los tipos de datos
dfBusinessYelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   business_id     150346 non-null  object 
 1   name            150346 non-null  object 
 2   address         145219 non-null  object 
 3   city            150346 non-null  object 
 4   state           150343 non-null  object 
 5   postal_code     150273 non-null  object 
 6   latitude        150346 non-null  float64
 7   longitude       150346 non-null  float64
 8   stars           150346 non-null  float64
 9   review_count    150346 non-null  int64  
 10  is_open         150346 non-null  int64  
 11  attributes      136602 non-null  object 
 12  categories      150243 non-null  object 
 13  hours           127123 non-null  object 
 14  business_id.1   5 non-null       object 
 15  name.1          5 non-null       object 
 16  address.1       5 non-null       object 
 17  city.1    

In [6]:
# Contar filas vacías por columna
filas_vacias_por_columna = dfBusinessYelp.isnull().sum()
print(filas_vacias_por_columna)

business_id            0
name                   0
address             5127
city                   0
state                  3
postal_code           73
latitude               0
longitude              0
stars                  0
review_count           0
is_open                0
attributes         13744
categories           103
hours              23223
business_id.1     150341
name.1            150341
address.1         150341
city.1            150341
state.1           150341
postal_code.1     150341
latitude.1        150341
longitude.1       150341
stars.1           150341
review_count.1    150341
is_open.1         150341
attributes.1      150341
categories.1      150341
hours.1           150341
dtype: int64


Al evaluar la cantidad de nulos por columnas vemos que las columnas : 'business_id.1', 'name.1',
       'address.1', 'city.1', 'state.1', 'postal_code.1', 'latitude.1',
       'longitude.1', 'stars.1', 'review_count.1', 'is_open.1', 'attributes.1',
       'categories.1', 'hours.1', estan llenos de nulos

In [7]:
# Contar la cantidad de duplicados del DataFrame
filas_duplicadas = dfBusinessYelp.duplicated().sum()
print("Filas duplicadas en el DataFrame:", filas_duplicadas)

Filas duplicadas en el DataFrame: 0


Como logramos observas no encontramos valores duplicados en el DataFrame

In [8]:
dfBusinessYelp['state'].unique()

array([nan, 'CA', 'MO', 'AZ', 'PA', 'TN', 'FL', 'IN', 'LA', 'AB', 'NV',
       'ID', 'DE', 'IL', 'NJ', 'NC', 'CO', 'WA', 'HI', 'UT', 'TX', 'MT',
       'MI', 'SD', 'XMS', 'MA', 'VI', 'VT'], dtype=object)

Al ver que hay valores nulos, siglas de los diferentes estados concluyo que la columna 'state' es mejor no usarla y sacar los estados mediante la longitud y latitud

### 3.1.2. Limpieza y Normalización de datos

#### Filtrado de datos

In [9]:
# El alcance del proyecto se limita a los diferentes negocios 
# dentro de la categoria 'Nail Salon', por lo que se aplica el filtro.

# Eliminar filas con valores NaN en la columna 'categories'
dfBusinessYelp = dfBusinessYelp.dropna(subset=['categories'])

# Filtrar las filas que contienen 'Nail Salons' en la columna 'categories'
dfBusinessYelp = dfBusinessYelp[dfBusinessYelp['categories'].str.contains('Nail Salons')]


De limitamos el proyecto basado en la Categoria 'Nail Salon' o 'Salon de Uñas'

In [10]:
dfBusinessYelp.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state.1,postal_code.1,latitude.1,longitude.1,stars.1,review_count.1,is_open.1,attributes.1,categories.1,hours.1
74,9Rww8yE6Dm4dSMEq09nwXg,Holly Nails & Spa,9101 Belcher Rd,Pinellas Park,TN,33782,27.855347,-82.736308,4.5,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,my0bmPD5dgDFE1ia__LNlw,Nails & Beauty Lounge,953 E Brandon Blvd,Brandon,IN,33511,27.937166,-82.269959,3.5,84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,CLEWowfkj-wKYJlQDqT1aw,Blow Bar Express Styling Salon,"310 S Dale Mabry Hwy, Ste 110",Tampa,TN,33609,27.941975,-82.505307,4.0,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Limpieza y Normalizacion

In [11]:
columnas_a_eliminar = ['is_open', 'categories','state', 'attributes', 
       'business_id.1', 'name.1', 'address.1', 'city.1', 'state.1', 
       'postal_code.1', 'latitude.1', 'longitude.1', 'stars.1', 'review_count.1',
        'is_open.1', 'attributes.1', 'categories.1', 'hours.1']

# Eliminar las columnas especificadas
dfBusinessYelp = dfBusinessYelp.drop(columnas_a_eliminar, axis=1)

Como mencionamos en el conteo de nulos eliminamos columnas con demasiados valores vacios que no nos ayudaran para realizar un buen analisis.
Las columnas'is_open','address', 'name' ya tenemos los valores y se filtro de manera correcta asi que seria redundante tener esas columnnas.
Con respecto a la columna 'state' tiene valores que no corresponde con la latitud y longitud en el mapa, asi que mejor es no contar con esa columna.

In [12]:
# Define una función para obtener el estado desde la latitud y longitud
def reverse_geocode(lat, lon):
    # Aquí va tu código para obtener el estado desde la latitud y longitud
    url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}"
    response = requests.get(url)
    data = response.json()
    if 'address' in data:
        state = data['address'].get('state', '')
        return state

# Aplica la función a cada fila del DataFrame y crea una lista de resultados
states = [reverse_geocode(row['latitude'], row['longitude']) for _, row in dfBusinessYelp.iterrows()]

# Asigna la lista de estados a la columna 'state' del DataFrame
dfBusinessYelp['state'] = states

Usando la API OpenStreetMap (OSM) se realizo una geocodificacion inversa que me devuelve los diferentes estados('state') basado en las columnas 'latitude' y 'longitude' 

In [13]:
dfBusinessYelp['state'].value_counts(10)

state
Pennsylvania    0.249946
Florida         0.189786
New Jersey      0.075958
Indiana         0.072495
Tennessee       0.070548
Missouri        0.066436
Arizona         0.062108
Louisiana       0.058429
Nevada          0.040467
Alberta         0.032244
Idaho           0.028998
Delaware        0.019693
California      0.017961
Illinois        0.014932
Name: proportion, dtype: float64

Vemos que estado tienes mas establecimientos

In [14]:
# Aplicamos el filtro con referencia en la columna 'name'
dfBusinessYelp = dfBusinessYelp[dfBusinessYelp['state'] == 'Florida']

Delimitamos los datos a un solo estado 'Florida'

In [15]:
dfBusinessYelp = dfBusinessYelp.drop('state', axis=1)

Borramos el estado 'Florida' debido que todos los salones de uña 'Nail Salon' pertenecen al mismo estado

In [16]:
dfBusinessYelp.head(2)

,business_id,name,address,city,postal_code,latitude,longitude,stars,review_count,hours
74,9Rww8yE6Dm4dSMEq09nwXg,Holly Nails & Spa,9101 Belcher Rd,Pinellas Park,33782,27.855347,-82.736308,4.5,16,"{'Tuesday': '9:30-19:0', 'Wednesday': '9:30-19..."
250,my0bmPD5dgDFE1ia__LNlw,Nails & Beauty Lounge,953 E Brandon Blvd,Brandon,33511,27.937166,-82.269959,3.5,84,"{'Monday': '9:30-19:0', 'Wednesday': '9:30-19:..."


In [17]:
#nuevo_orden_columnas = ['business_id','name', 'address', 'city', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count','hours','source']

#dfBusinessYelp = dfBusinessYelp[nuevo_orden_columnas]

Reorganizamos la columnas para tener un mismo orden con la tabla de Google

In [18]:
# Se creó la columna 'source' con un valor unico 'Y' 
dfBusinessYelp['source']='Y'

Se creo la colummna 'source' para identificar la tabla que pertenecen a los datos obtenidos de las tablas Yelp

In [19]:
dfBusinessYelp.head()

,business_id,name,address,city,postal_code,latitude,longitude,stars,review_count,hours,source
74,9Rww8yE6Dm4dSMEq09nwXg,Holly Nails & Spa,9101 Belcher Rd,Pinellas Park,33782,27.855347,-82.736308,4.5,16,"{'Tuesday': '9:30-19:0', 'Wednesday': '9:30-19...",Y
250,my0bmPD5dgDFE1ia__LNlw,Nails & Beauty Lounge,953 E Brandon Blvd,Brandon,33511,27.937166,-82.269959,3.5,84,"{'Monday': '9:30-19:0', 'Wednesday': '9:30-19:...",Y
294,CLEWowfkj-wKYJlQDqT1aw,Blow Bar Express Styling Salon,"310 S Dale Mabry Hwy, Ste 110",Tampa,33609,27.941975,-82.505307,4.0,55,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-16:0', '...",Y
362,gbetDRjllP5cOxaWxFjGHA,Pretty Nails,5316 Little Rd,New Port Richey,34655,28.237112,-82.668024,3.0,6,NaN,Y
630,FYzZd2mlMIPScy95UY2xkw,McMahan Hair Salon,10464 Roosevelt Blvd N,St. Petersburg,33716,27.866938,-82.645031,4.0,11,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-20:0', '...",Y


## 3.2 Dataset Reviews Yelp

### 3.2.1. Exploración inicial de los datos

In [20]:
#Vizualizo las primeras filas del DataFrame
dfReviewYelp.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,2018
1,lUUhg8ltDsUZ9h0xnwY4Dg,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,4,1,0,1,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07,2018
2,JBWZmBy69VMggxj3eYn17Q,aFa96pz67TwOFu4Weq5Agg,kq5Ghhh14r-eCxlVmlyd8w,5,0,0,0,My boyfriend and I tried this deli for the fir...,2018-08-23 21:39:38,2018
3,E9AB7V4z8xrt2uPF7T55FQ,iYY5Ii1LGpZCpXFkHlMefw,Zx7n8mdt8OzLRXVzolXNhQ,5,0,0,0,Amazing biscuits and (fill in the blank). Grea...,2018-04-27 23:03:21,2018
4,A4n4YaE-owOVgTQcrVqHUw,S7bjj-L07JuRr-tpX1UZLw,I6L0Zxi5Ww0zEWSAVgngeQ,4,0,0,0,The cafe was extremely cute. We came at 8am an...,2018-07-07 20:50:12,2018


In [21]:
dfReviewYelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3018057 entries, 0 to 3018056
Data columns (total 10 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_id    object
 1   user_id      object
 2   business_id  object
 3   stars        int64 
 4   useful       int64 
 5   funny        int64 
 6   cool         int64 
 7   text         object
 8   date         object
 9   year         int64 
dtypes: int64(5), object(5)
memory usage: 230.3+ MB


In [22]:
# Contar filas vacías por columna
filas_vacias_por_columna = dfReviewYelp.isnull().sum()
print(filas_vacias_por_columna)

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
year           0
dtype: int64


Como observamos no tenemos valores nulos

In [23]:
# Contar la cantidad de duplicados del DataFrame
filas_duplicadas = dfReviewYelp.duplicated().sum()
print("Filas duplicadas en el DataFrame:", filas_duplicadas)

Filas duplicadas en el DataFrame: 0


Como observamos no tenemos filas duplicadas

### 3.2.2. Limpieza y Normalización de datos

Filtrado de datos

In [24]:
# Obtener los id únicos del DataFrame dfbusinessYelp
business_ids_to_keep = dfBusinessYelp['business_id'].unique()


# Friltamos el dfreviewYelp basado en los balores unico de lao id de dfbusinessYelp
dfReviewYelp = dfReviewYelp[dfReviewYelp['business_id'].isin(business_ids_to_keep)]

# El conjunto de datos se filtró por la fecha desde el inicio del 2016 hasta finalizar el 2021.
#dfreviewYelp = dfreviewYelp[(dfreviewYelp['date'] >= '2016-01-01') & (dfreviewYelp['date'] >= '2021-12-31')]

Aplicamos un filtro solo para quedarnos con los valores que coincidan con el id del negocio

#### Limpieza y Normalizacion

In [25]:
# Convertir la columna 'date' a tipo datetime
dfReviewYelp['date'] = pd.to_datetime(dfReviewYelp['date'])

In [26]:
columnas_a_eliminar = ['review_id', 'useful', 'funny', 'cool']

# Eliminar las columnas especificadas
dfReviewYelp = dfReviewYelp.drop(columnas_a_eliminar, axis=1)

Elinamos las siguientes columnas 'useful', 'funny', 'cool' debido que no tiene una guia o rango que me diga si esta bien o no, 'date' lo dividimos en años , mes, dia asi que no lo necesitamos

In [27]:
# Aplicar la limpieza de texto directamente sobre la columna 'text'
dfReviewYelp['text'] = dfReviewYelp['text'].str.lower().str.replace(r'[^a-z0-9\s]', '')

Convertimos todo los valores de la columna 'text' a minuscula para guiarnos mejor

In [28]:
# Modificamos el tipo de dato a un valor flotante
dfReviewYelp['stars'] = dfReviewYelp['stars'].astype('float64')

Transforme la columna 'stars' para tener similitud del tipo de dato con otra tabla

In [29]:
# Se añade la columna 'source', con valor unico 'Y'
dfReviewYelp['source']='Y'

Se añade la columna 'source' y se usa 'Y' para indicar que los datos provienen del conjunto de datos de Yelp.

In [30]:
dfReviewYelp = dfReviewYelp.rename(columns={'stars' : 'review_stars'})

Renombrom stars para indentificar a que se refiere esa calificacion

In [31]:
dfReviewYelp.head(3)

,user_id,business_id,review_stars,text,date,year,source
51,6Mhbp06FEhD9uxUGCCUmKA,fBNjTSdPJ1mtTzIHYpQnDw,1.0,zero stars!! this place is dirty and disorgani...,2018-09-03 13:04:03,2018,Y
96,-FFgJuDZLmcZtrxDTRC7sg,my0bmPD5dgDFE1ia__LNlw,5.0,this is the place we have been searching for! ...,2018-08-23 22:28:46,2018,Y
556,g34LsuGf9gj9h5wqhye-Jw,bP8Y5cHMda64Z3tGA-cYzw,5.0,i found this place through yelp. i was attract...,2018-03-23 19:29:16,2018,Y


## 4. Union y Almacenamiento de Datos

In [32]:
dfBusinessYelp.to_csv('DataSets/Cleaned_Business_Y.csv', index=False)
dfReviewYelp.to_csv('DataSets/Cleaned_Review_Y.csv', index= False)

In [33]:
# Unir las tablas por la columna 'business_id'
df_Yelp = pd.merge(dfBusinessYelp, dfReviewYelp, on='business_id', how='inner')

In [34]:
df_Yelp.to_csv('DataSets/Nail_Salon_Yelp.csv', index =False)

In [35]:
df_Yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15591 entries, 0 to 15590
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   business_id   15591 non-null  object        
 1   name          15591 non-null  object        
 2   address       15568 non-null  object        
 3   city          15591 non-null  object        
 4   postal_code   15591 non-null  object        
 5   latitude      15591 non-null  float64       
 6   longitude     15591 non-null  float64       
 7   stars         15591 non-null  float64       
 8   review_count  15591 non-null  int64         
 9   hours         14426 non-null  object        
 10  source_x      15591 non-null  object        
 11  user_id       15591 non-null  object        
 12  review_stars  15591 non-null  float64       
 13  text          15591 non-null  object        
 14  date          15591 non-null  datetime64[ns]
 15  year          15591 non-null  int64 